In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader

# 定义前馈神经网络模型
class FeedforwardNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(FeedforwardNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)
        
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out



## 分别实现momentum、rmsprop、adam优化器

In [3]:
def init_momentum_states(params):
    v_w1,v_b1,v_w2,v_b2 = torch.zeros(params[0].shape),torch.zeros(params[1].shape),torch.zeros(params[2].shape),torch.zeros(params[3].shape)
    return (v_w1,v_b1,v_w2,v_b2)
def sgd_momentum(params,states,lr,momentum):
    for p,v in zip(params,states):
        with torch.no_grad():
            v[:]=momentum*v - p.grad
            p[:]+=lr*v
        p.grad.data.zero_()

In [8]:
def init_rmsprop_states(params):
    s_w1,s_b1,s_w2, s_b2=torch .zeros(params[0].shape),
    torch.zeros(params[1].shape),
    torch.zeros(params[2].shape),
    torch.zeros(params[3].shape)
    return (s_w1, s_b1, s_w2, s_b2)

def rmsprop(params, states, lr,gamma):
    gamma, eps = gamma , 1e-6
    for p, s in zip(params, states):
        with torch.no_grad():
            s[:] = gamma * s + (1 - gamma) * torch.square(p.grad)
            p[:] -= Ir * p.grad / torch.sqrt(s + eps)
        p.grad.data.zero_()

In [9]:
def init_adam_states(params):
    v_w1,v_b1,v_w2,v_b2=torch.zeros(params[0].shape)
    ,torch.zeros(params[1].shape)
    ,torch.zeros(params[2].shape)
    ,torch.zeros(params[3].shape)
    s_w1,s_b1,s_w2,s_b2 = torch.zeros(params[0].shape)
    ,torch.zeros(params[1].shape)
    ,torch.zeros(params[2].shape)
    ,torch.zeros(params[3].shape)
    return ((v_w1,s_w1),(v_b1,s_b1),(v_w2,s_w2),(v_b2,s_b2))
def adam(params,states,lr,t):
    beta1,beta2,eps = 0.9,0.999,1e-6
    t+=1#更新迭代次数
    
    for p,(v,s) in zip(params,states):
        with torch.no_grad():
        #补全  
            # 更新动量和二阶动量
            v.data = beta1 * v + (1 - beta1) * p.grad.data
            s.data = beta2 * s + (1 - beta2) * p.grad.data**2
            # 进行偏差修正
            v_hat = v / (1 - beta1**t)
            s_hat = s / (1 - beta2**t)
            # 更新参数
            p.data -= lr * v_hat / (torch.sqrt(s_hat) + eps)
        p.grad.data.zero_()
    

In [3]:
# 定义超参数
input_size = 28 * 28  # 输入特征的维度
hidden_size = 128  # 隐藏层的大小
num_classes = 10  # 类别的数量
learning_rate = 0.001  # 学习率
num_epochs = 10  # 迭代次数
batch_size = 64  # 批次大小



In [4]:
# 加载数据集
train_dataset = MNIST(root='~/Datasets/MNIST', train=True, transform=ToTensor(), download=True)
test_dataset = MNIST(root='~/Datasets/MNIST', train=False, transform=ToTensor(), download=False)

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

# 初始化模型
model = FeedforwardNN(input_size, hidden_size, num_classes)

# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)



In [ ]:
# 训练模型
for epoch in range(num_epochs):
    model.train()
    for batch_idx, (data, targets) in enumerate(train_loader):
        data = data.view(-1, input_size)
        
        # 前向传播
        outputs = model(data)
        loss = criterion(outputs, targets)
        
        # 反向传播和优化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (batch_idx+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{batch_idx+1}/{len(train_loader)}], Loss: {loss.item():.4f}')
    
    # 在测试集上评估模型
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for data, targets in test_loader:
            data = data.view(-1, input_size)
            outputs = model(data)
            _, predicted = torch.max(outputs.data, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()
        
        accuracy = correct / total
        print(f'Test Accuracy: {accuracy:.4f}')


Epoch [1/10], Step [100/938], Loss: 2.2894
Epoch [1/10], Step [200/938], Loss: 2.2859
Epoch [1/10], Step [300/938], Loss: 2.2415


In [2]:
# 保存模型
torch.save(model.state_dict(), 'multi_class_model2.pth')

Epoch [1/10], Step [100/938], Loss: 0.5146
Epoch [1/10], Step [200/938], Loss: 0.6982
Epoch [1/10], Step [300/938], Loss: 0.5684
Epoch [1/10], Step [400/938], Loss: 0.5628
Epoch [1/10], Step [500/938], Loss: 0.7565
Epoch [1/10], Step [600/938], Loss: 0.5858
Epoch [1/10], Step [700/938], Loss: 0.6392
Epoch [1/10], Step [800/938], Loss: 0.5270
Epoch [1/10], Step [900/938], Loss: 0.4213
Test Accuracy: 0.8700
Epoch [2/10], Step [100/938], Loss: 0.5837
Epoch [2/10], Step [200/938], Loss: 0.6294
Epoch [2/10], Step [300/938], Loss: 0.5749
Epoch [2/10], Step [400/938], Loss: 0.6435
Epoch [2/10], Step [500/938], Loss: 0.5448
Epoch [2/10], Step [600/938], Loss: 0.6444
Epoch [2/10], Step [700/938], Loss: 0.6073
Epoch [2/10], Step [800/938], Loss: 0.5115
Epoch [2/10], Step [900/938], Loss: 0.4689
Test Accuracy: 0.8743
Epoch [3/10], Step [100/938], Loss: 0.9086
Epoch [3/10], Step [200/938], Loss: 0.4388
Epoch [3/10], Step [300/938], Loss: 0.7367
Epoch [3/10], Step [400/938], Loss: 0.3972
Epoch [3/1